# Outil pour analyse des fichiers de sortie

Avant toute chose, il vous faut créer votre environnement avec conda.

L'avantage de créer un environnement spécifique est que vous serez assuré du bon fonctionnement de votre code dans le temps (malgré les mises à jour des packages) et sur toute machine en transmettant les caractéristiques de cet environnement. 

Nous vous proposons de créer votre environnement via `conda-forge` en s'appuyant sur le fichier dédié à vos projets: `environment_P07-10.yml`. Ce fichier donne un nom à l'environnement, ici : ITS_fromP07_toP09 et décrit l'ensemble des packages nécessaires et leur version. Pour créer votre environnement sur votre machine, il vous suffit d'exécuter les lignes suivantes sur votrer terminal de commande:

```
cd "your/path/to/the/folder/with/the/file/yml"
conda env create -f environment_P07-10.yml
conda info --envs
```

Votre environnement devrait être créé et la dernière ligne permet de vérifier qu'il existe bien parmi vos environnements.

Vous pouvez désormais l'activer via la commande : 
```
conda activate ITS_fromP07_toP10
```
et/ou le sélectionner lorsque vous lancez votre jupyterlab!


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os

pd.__version__

**Conseil :** Vérfier que votre version de panda est bien 1.3.5 au moins, sinon vous aurez des soucis lors de l'utilisation de la fonction pd.read_xml()!

# Définition du chemin vers votre répertoire de travail

La première étape consiste à définir votre dossier de travail, celui où vous allez stocker les sorties de la modélisation et procéder à l'analyse.

Par la suite, les chemins sont définis de façon relative.

In [ ]:
WorkingDirectory = "your/path/to/folder/TP_simulation/" # "C:/Users/eleon/OneDrive - entpe.fr/Documents/these/ITS/ITS/ITS/Projet_09_VSL/"
os.chdir(WorkingDirectory)
print(os.getcwd())

## 0. Format du fichier

### 0.1 Visulation des données

Le fichier trip_info.xml contient le résumé de la traversée du réseau par chaque véhicule.

In [ ]:
#'stats_bas' est le nom du 'outputFolder' aui a été défini au début du script Python TP_ITS.py
trip_info_analyses_0 = pd.read_xml("./stats_bas/tripinfo.xml")


In [ ]:
trip_info_analyses_0.head()

In [ ]:
print(trip_info_analyses_0.columns)

Prenons quelques indicateurs en exemple : 
(i) 'duration' correspond au temps de parcours total de chaque véhicule c'est à dire le temps qu'il a passé sur le réseau;
(ii) vtype correspond au type de véhicule, on en distingue 3 dans notre étude : les véhicules connectés ('CV'), les véhicules non connectés ('HDV') et les véhicules de type 'autre' qui correspondent aux véhicules de la voie d'insertion (uniquement simulés pour créer la congestion sur l'axe principal et donc non pris en compte dans l'analyse de nore réseau).

In [ ]:
trip_info_analyses_0.duration.describe()

In [ ]:
trip_info_analyses_0.vType

### 0.2 Préparation des données

Une des principales étapes de l'analyse des données consiste à préparer celles-ci en les mettant en forme, triant, etc.
(i) 'tri_colonne' de tri des données. Ici, on trie les données pour ne garder que les véhicules de type 'CV' et 'HDV' puisque les véhicules de type 'autre' de la voie sont uniquement générés pour créer la congestion sur l'axe principal et ne sont pas pris en compte pour les analyses. Egalement, on considère une période de chargement de 5 min pour éviter les effets de bords,  donc on ne considère que les véhicules dont le départ est supérieur à 5 min (300s).
(ii) 'grouped_trip_info' est un exemple de mise en forme des données.

In [ ]:
def tri_colonne(path):
    trip_info = pd.read_xml(path)
    print('number rows trip_info', len(trip_info.index))
    trip_info_tri = trip_info[trip_info.depart > 300]
    print('number rows trip_info_tri', len(trip_info_tri.index))
    trip_info_tri_2 = trip_info_tri[trip_info_tri.vType != "autre"]
    print('number rows trip_info_tri_2', len(trip_info_tri_2.index))
    
    
    print('Description des temps de parcours des véhicules avant le tri', trip_info.duration.describe())
    print('Description des temps de parcours des véhicules après le tri', trip_info_tri_2.duration.describe())
    return trip_info_tri_2
    

In [ ]:
path = "./stats_bas/tripinfo.xml"
trip_info_0 = tri_colonne(path)

In [ ]:
grouped_trip_info = trip_info_0.groupby(["vType"]).aggregate(
                nbVType = pd.NamedAgg(column="vType", aggfunc="count"),
                avgWaitingTime = pd.NamedAgg(column="waitingTime", aggfunc="mean"),
                avgTimeLoss = pd.NamedAgg(column="timeLoss", aggfunc="mean"),
                totalWaitingCount= pd.NamedAgg(column="waitingCount", aggfunc="sum")
            ).reset_index()
            
grouped_trip_info

## 1. Exemples d'indicateurs et d'outils pour observer l'efficacité du réseau

On utilise quelques outis de visualisation issus de la galerie plotly : https://plotly.com/python/. 

### 1.1 Indicateur: temps de parcours

In [ ]:
### box plot

fig = go.Figure()
fig.add_trace(go.Box(y=trip_info_0["duration"], name = "Baseline"))
fig.show()

On peut également tracer sur un graphe plusieurs informations (et par exemple comparer ensuite les informations de temps de parcours entre la situation de référence et une situation où on applique un système de régulation de trafic).

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=trip_info_0["duration"], name = "Baseline"))
fig.add_trace(go.Box(y=trip_info_0["duration"], name = "Alternative"))
fig.show()

In [ ]:
### histogramme

fig = go.Figure()
fig.add_trace(go.Histogram(x=trip_info_0["duration"], name = "Baseline"))
fig.update_layout(barmode = "overlay")
fig.update_traces(opacity = 0.5)
fig.show()

Dans les histogrammes, on peut montrer l'argument permettant d'observer des densités plutôt que des décomptes.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(y=trip_info_0["duration"], name = "Baseline", histnorm='probability density'))
fig.update_layout(barmode = "overlay")
fig.update_traces(opacity = 0.5)
fig.show()

In [ ]:
print(trip_info_0.duration.describe())

On peut aussi utiliser nos données pour créer un nouvel indicateur (peut-être plus explicite), par exemple ici le Temps Total Passé par tous les véhicules sur le réseau.

In [ ]:
print('Total Time Spent (h)', trip_info_0["duration"].sum()/3600)


In [ ]:
ar = np.array([[trip_info_0["duration"].sum()/3600]])
df = pd.DataFrame(ar, index = ['Baseline'], columns = ['Total Time Spent '])
print(df)   

### 1.2 Indicateur: vitesse moyenne

In [ ]:
path_traj = "./stats_bas/fcdStats.xml"
traj_analyses_0 = pd.read_xml(path_traj)
traj_analyses_0.head()

Le lecteur (parser) de Pandas n'est pas en mesure de lire les structures complexes des fichiers XML, il est donc nécessaire de convertir le fichier .xml en fichier .csv.

Il existe une fonction de SUMO permettant de convertir les fichiers XML en CSV, il faut taper la commande suivante dans votre terminal: 'python xml/xml2csv.py ./stats_bas/fcdStats.xml'.
Pour plus d'informations, vous pouvez aller voir la documentation de SUMO : https://sumo.dlr.de/docs/Tools/Xml.html.

In [ ]:
path_fcd_csv = "./stats_bas/fcdStats.csv"
fcd_analyses_0 = pd.read_csv(path_fcd_csv, sep = ';')
fcd_analyses_0.head()

In [ ]:
print(fcd_analyses_0.columns)

In [ ]:
fcd_0_tri = fcd_analyses_0[fcd_analyses_0.timestep_time > 300]
number_rows = len(fcd_analyses_0.index)
print(number_rows)
number_rows_tri = len(fcd_0_tri.index)
print(number_rows_tri)

fcd_0_tri_2 = fcd_0_tri[fcd_0_tri.vehicle_type != "autre"]
print(len(fcd_0_tri_2.index))

In [ ]:
## Groupe les vehicles dans le même pas de temps et fait la moyenne
fcd_0 = fcd_0_tri_2.groupby('timestep_time',as_index=False)
fcd_mean_0=fcd_0.mean()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=fcd_mean_0["timestep_time"].tolist(), y=fcd_mean_0["vehicle_speed"].tolist() ,name = 'Baseline mean speed'))

fig.show()

### 1.3 Indicateur: trajectoire des véhicules

Les diagrammes espace-temps permettent une bonne visualistaion de la congestion ou de l'impact des stratégies de régulation sur le réseau.

a) Diagramme espace-temps: les deux voies confondues

In [ ]:
#det
print(len(fcd_0_tri_2.index))
index_with_nan = fcd_0_tri_2.index[fcd_0_tri_2.isnull().any(axis=1)]
fcd_0_tri_2.drop(index_with_nan,0, inplace=True)
print(len(fcd_0_tri_2.index))

fig = px.scatter(fcd_0_tri_2, x='timestep_time', y='vehicle_pos', color='vehicle_speed', color_continuous_scale='jet_r' )
fig.show()

b) Diagramme espace-temps par voie

In [ ]:
fcd_0_tri_2_lane0 = fcd_0_tri_2[fcd_0_tri_2["vehicle_lane"]=='E_0']
fig = px.scatter(fcd_0_tri_2_lane0, x='timestep_time', y='vehicle_pos', color='vehicle_speed', color_continuous_scale='jet_r')
fig.show()

fcd_0_tri_2_lane1 = fcd_0_tri_2[fcd_0_tri_2["vehicle_lane"]=='E_1']
fig = px.scatter(fcd_0_tri_2_lane1, x='timestep_time', y='vehicle_pos', color='vehicle_speed', color_continuous_scale='jet_r' )
fig.show()


## 2. Exemples d'indicateurs pour observer l'impact environnmental du réseau

### 2.1 Indicateur: émissions CO2

In [ ]:
path_tripinfo_csv = "./stats_bas/tripinfo.csv"
trip_csv_0 = pd.read_csv(path_tripinfo_csv, sep = ';')
print(trip_csv_0.head())

In [ ]:
print(trip_csv_0.columns)

In [ ]:
print(len(trip_csv_0.index))
trip_tri_1 = trip_csv_0[trip_csv_0.tripinfo_depart > 300]
print(len(trip_tri_1))
trip_tri_2 = trip_tri_1[trip_tri_1.tripinfo_vType != "autre"]
print(len(trip_tri_2))

In [ ]:
trip_tri_2.emissions_CO2_abs.describe()

In [ ]:
trip_tri_2.emissions_CO2_abs.sum()

## 3. Exemples d'indicateurs pour observer la sécurité sur le réseau

### 3.1 Indicateur: homogénéisation des temps de parcours entre les véhicules 

On calcule le coefficient de variation entre les temps de parcours des véhicules. Un faible coefficient de variation indique une faible variation entre les temps de parcours des véhicuels et donc une homogénisation de la circulation sur le réseau. Au contraire, un fort coefficient de variation indique une forte hétérogénéité de comportement entre les véhicules sur le réseau.

Ici, il n'a de sens que si on le compare à un autre coefficient de variation (par exemple un coefficient de variation d'une simulation où on applique un systme de régulation de trafic).

In [ ]:
meanTTS = trip_tri_2.tripinfo_duration.mean()
ecartTTS = trip_tri_2.tripinfo_duration.std()

print( 'coefficient de variation des temps de parcours de la situation de référence', ecartTTS/meanTTS)